In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
import json
import time

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Import API key
from keys import api_key

#State Abbr
stateAbbr_file = "Resources/StateAbbr.csv"

#Year to pull data
year = 2017

In [ ]:
#Read purchasing file and store into Pandas DF
state_abbr = pd.read_csv(stateAbbr_file)
state_abbr.head()

In [ ]:
# create an empty dataframe, to store data retrived by API call
crimeDataAll_df = pd.DataFrame()

#Loop through each state, to get the crime data
for state in state_abbr["State Abbr"]:
    print(f"Processing State: {state}")
    api_url = f"https://api.usa.gov/crime/fbi/sapi/api/summarized/estimates/states/{state}/{year}/{year}?API_KEY=" + api_key
    
    # make the API call
    crimeData_json = requests.get(api_url).json()  
    
    # skip the state, if the API fails. 
    try:
        stateCrime_df = pd.DataFrame(crimeData_json["results"])
        
        #if Dataframe is empty, copy the assign, else append the df
        if crimeDataAll_df.empty:
            crimeDataAll_df = stateCrime_df
        else:
            crimeDataAll_df = pd.concat([crimeDataAll_df, stateCrime_df], ignore_index=True)
        
    except (KeyError, IndexError):
        print(f'Skippinh "{state}". data for state not exist.')
    
print(f"Completed all states")
# Out of State For loop

# Preview dataframe
crimeDataAll_df

In [ ]:
# Filter data for year
crimeDataOffense_df = crimeDataAll_df[crimeDataAll_df["year"] == year]
crimeDataOffense_df

In [ ]:
# calculate total offense, summing each offense category
crimeDataOffense_df["total offense"] = crimeDataOffense_df["violent_crime"] + crimeDataOffense_df["homicide"] \
                                        + crimeDataOffense_df["rape_revised"] + crimeDataOffense_df["robbery"] \
                                        + crimeDataOffense_df["aggravated_assault"] + crimeDataOffense_df["property_crime"] \
                                        + crimeDataOffense_df["burglary"] + crimeDataOffense_df["larceny"] \
                                        + crimeDataOffense_df["motor_vehicle_theft"] + crimeDataOffense_df["arson"]
# preview dataframe
crimeDataOffense_df

In [ ]:
# Create Crime Data frame with only the needed column
crimeData_df = crimeDataOffense_df[["state_abbr","year","population","total offense"]]
crimeData_df = crimeData_df.rename(columns={"state_abbr":"state"})
crimeData_df

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
crimeData_df.to_csv("Resources/CrimeData.csv", index=False, header=True)